In [1]:
import os

In [2]:
%pwd

'c:\\Users\\670258675\\Music\\New Folder\\Projects\\NLP---Text-Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\670258675\\Music\\New Folder\\Projects\\NLP---Text-Summarizer'

In [5]:
## Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        # create root folder
        create_directories([self.config.artifacts_root])
        
    # we have prepared the return type above (DataIngestionConfig)
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [8]:
## For Components
import os
import urllib.request as request
import zipfile
from src.textSummarizer.logging import logger
from src.textSummarizer.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    # download the data from URL
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file ## where I want to download.
            )
            logger.info(f"{filename} downloaded! with the following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    # extract the zip file
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
## Create Pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-17 13:06:21,417: INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-03-17 13:06:21,423: INFO:common:yaml file: params.yaml loaded successfully]
[2024-03-17 13:06:21,428: INFO:common:Creating directory at: artifacts]
[2024-03-17 13:06:21,432: INFO:common:Creating directory at: artifacts/data_ingestion]


[2024-03-17 13:08:08,949: INFO:2322604614:artifacts/data_ingestion/data.zip downloaded! with the following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: D2C6:173E00:FB948:166FA9:65F69BEB
Accept-Ranges: bytes
Date: Sun, 17 Mar 2024 07:38:01 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100031-HYD
X-Cache: HIT
X-Cache-Hits: 0
X-Timer: S1710661081.343141,VS0,VE1
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 0f1b98468d4aec1ec733cbfe4e31e2c4fb224605
Expires: Sun, 17 Mar 2024 07:43:01 GMT
Source-Age: 69

]
